In [1]:
import torch
import torch.nn.functional as functional
from torchvision import transforms
from models import resnet50
from accessory_functions import dicom_path_to_tensor
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import cv2

normal_path = "/data/chd/data_subsets/sa_data/normal/"
svp_path = "/data/chd/data_subsets/sa_data/svp/"
normal_pids = [] #will be list of normal pids
for pid in os.listdir(normal_path):
    normal_pids.append(pid)
svp_pids = [] #will be list of svp pids
for pid in os.listdir(svp_path):
    svp_pids.append(pid)
random.seed(10)
test_svp_pids = random.sample(svp_pids,6) #each PID should be a full path to that PID directory
test_normal_pids = random.sample(normal_pids,9)
notest_normal_pids = list(set(normal_pids) - set(test_normal_pids))
notest_svp_pids = list(set(svp_pids) - set(test_svp_pids))
random.shuffle(notest_normal_pids)
random.shuffle(notest_svp_pids)
svp_folds = []
normal_folds = []
svp_fold_size = len(notest_svp_pids) // 4
normal_fold_size = len(notest_normal_pids) // 4
for i in range(4):
    if i != 3:
        svp_folds.append(notest_svp_pids[i*svp_fold_size:(i+1)*svp_fold_size])
        normal_folds.append(notest_normal_pids[i*normal_fold_size:(i+1)*normal_fold_size])
    elif i == 3:
        svp_folds.append(notest_svp_pids[i*svp_fold_size:])
        normal_folds.append(notest_normal_pids[i*normal_fold_size:])

# Load the model
model = resnet50(pretrained=True)
model_weight_path = "./saved_models/resnet50_fold0_pretrained.pth"
model.load_state_dict(torch.load(model_weight_path))
model.eval()

#find best-performing PID
analysis_csv = pd.read_csv("analysis_csvs/resnet50_fold0_pretrained.csv")
test_svp_csv = analysis_csv[analysis_csv["pid"] == test_svp_pids[0]]
test_example = test_svp_csv.iloc[0]
example_img_path = test_example['path']
# print(test_example)
# print(example_img_path)
# for date in test_svp_csv['date'].unique():
#     date_csv = test_svp_csv[test_svp_csv['date'] == date]
#     z_csv = date_csv[date_csv['z_coord'] == -19.311980]
#     t_csv = z_csv[z_csv['time'] == z_csv['time'].min()]
#     break
# example_img_path = t_csv['path'].item()

# Load the image
img = dicom_path_to_tensor(example_img_path,224,train=False)
img_input = img.unsqueeze(0)

/home/cjmorris/miniconda3/envs/deep_learning/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cjmorris/miniconda3/envs/deep_learning/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Perform a forward pass to get the model's predictions
output = model(img_input)
predicted_class = output.argmax(dim=1).item()

# Get the gradients of the predicted class with respect to the feature maps of the last convolutional layer
model.zero_grad()
output[0, predicted_class].backward()

# Get the gradients and the feature maps
gradients = model.layer4[2].conv3.weight.grad
feature_maps = model.layer4[2].conv3.weight

# Compute the guided gradients
guided_gradients = torch.mean(gradients, dim=[0, 2, 3])

# Compute the heatmap
heatmap = torch.mean(guided_gradients * feature_maps.squeeze(), dim=1).squeeze()
# heatmap = torch.mean(heatmap, axis=0) #custom addition
heatmap = functional.relu(heatmap)
heatmap /= torch.max(heatmap)

# Convert the heatmap to a numpy array
heatmap = heatmap.detach().numpy()

# Display the heatmap
plt.matshow(heatmap)
plt.show()

# Superimpose the heatmap on the original image
heatmap = np.uint8(255 * heatmap)
heatmap = Image.fromarray(heatmap).resize((img.size[0], img.size[1]), Image.ANTIALIAS)
heatmap = np.array(heatmap)
superimposed_img = np.array(img) * 0.6 + heatmap[:, :, np.newaxis] * 0.4
superimposed_img = Image.fromarray(np.uint8(superimposed_img))
superimposed_img.show()

In [2]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        # Register hooks for gradients and activations
        target_layer.register_forward_hook(self.forward_hook)
        target_layer.register_full_backward_hook(self.full_backward_hook)

    def forward_hook(self, module, input, output):
        self.activations = output.detach()

    def full_backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def compute_heatmap(self, input_batch, class_idx=None):
        # Forward pass
        logits = self.model(input_batch)
        self.model.zero_grad()

        if class_idx is None:
            class_idx = torch.argmax(logits, dim=1).item()

        # Compute gradients for the target class
        one_hot_output = torch.zeros_like(logits)
        one_hot_output[0, class_idx] = 1
        logits.backward(gradient=one_hot_output)

        # Compute Grad-CAM heatmap
        weights = torch.mean(self.gradients, dim=[2, 3], keepdim=True)
        heatmap = torch.sum(weights * self.activations, dim=1, keepdim=True)
        heatmap = torch.relu(heatmap)  # ReLU removes negative values
        heatmap /= torch.max(heatmap)  # Normalize to [0, 1]

        # Get the predicted class probability
        probs = torch.softmax(logits, dim=1)
        predicted_prob = probs[0, class_idx].item()

        return heatmap.squeeze().cpu().numpy(), class_idx, predicted_prob

In [ ]:
gradcam = GradCAM(model, model.layer4[2].conv3)
heatmap, predicted_class_idx, predicted_prob = gradcam.compute_heatmap(img_input)

In [ ]:
def visualize_heatmap(img: np.ndarray, heatmap: np.ndarray) -> None:
    # Read the image from the given file path
    # img = cv2.imread(img_path)
    
    # Resize the heatmap to match the size of the original image
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Normalize the heatmap values to the range [0, 255] and cast to uint8
    heatmap = np.uint8(255 * heatmap)
    
    # Apply the JET colormap to the heatmap
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Blend the original image with the heatmap (60% original, 40% heatmap)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    # Display the blended image in RGB format
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

torch.Size([1, 224, 224])

In [ ]:
visualize_heatmap(img,heatmap)

NameError: name 'cv2' is not defined

In [1]:
# test analysis csvs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('analysis_csvs/resnet50_fold0_pretrained.csv')

In [5]:
analysis_csv = pd.read_csv("analysis_csvs/resnet50_fold0_pretrained.csv")

In [25]:
gradients.shape
feature_maps.shape

In [45]:
test_heatmap = guided_gradients * feature_maps.squeeze()

In [48]:
torch.mean(test_heatmap,dim=1).shape

In [44]:
feature_maps.squeeze().shape

In [35]:
guided_gradients.shape
heatmap.shape

In [20]:
analysis_csv['z_coord'].median()

In [9]:
sorted(date_csv['z_coord'].unique())

In [49]:
np.sqrt(2048)

In [2]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model.layer4[2

SyntaxError: invalid syntax (2374418759.py, line 1)